# Capsule Network Model for detecting Covid-19

In [ ]:
import numpy as np
from keras import layers, models
from keras import backend as K
from capsulelayers import CapsuleLayer, PrimaryCap
from keras import utils as np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

K.set_image_data_format('channels_last')



def CapsNet(input_shape, n_class):
    """
    A Capsule Network on Capsule Network.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    # Layer 1a: Convolutional layer for detecting the edges from the image
    # Layer 1b: Relu layer for adding a non-linearity
    conv1 = layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    # Layer 2a: 6*6*32
    # Layer 2b: Squashing
    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')
    
    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16,
                             name='digitcaps')(primarycaps)

    
    #Flattening
    A = layers.Flatten()(digitcaps)

    #Full connection
    B = layers.Dense(32, input_shape = (2,), activation = 'relu')(A)
    C = layers.Dense(n_class, activation = 'relu')(B)
    D = layers.Softmax()(C)
    

    
    # Models for training and evaluation (prediction)
    train_model = models.Model([x], D)
    
    
    return train_model

# Generating the dataset
## 0 - Positive
## 1 - Negative

In [ ]:
def load_data():
    # grab the list of images in our dataset directory, then initialize
    # the list of data (i.e., images) and class images
    x_train = np.load('Covid.npy')
    x_train = x_train.reshape(-1,224,224,1)
    label = np.ones((50,),dtype = int)
    
    label[0:25] = 0
    label[25:50] = 1
    
    n_classes = 2
    
    train_data = [x_train, label]
    (x, y) = (train_data[0], train_data[1])
    y = np_utils.to_categorical(y, n_classes)
    X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=42 )
    
    return (X_train, y_train), (X_test, y_test)

# Training

In [ ]:
if __name__ == "__main__":
    # load data
    (X_train, y_train), (X_test, y_test) = load_data()
    input_shape = (224, 224, 1)
    batch_size = 10
    epochs = 25
    n_class = 2
    validation_data = (X_train, y_train)
    # define model
    model = CapsNet(input_shape=input_shape, n_class=n_class)
    model.summary()
    
    model.compile(optimizer='adam',
                  loss = 'mean_squared_error',
                  metrics=['accuracy'])
    
    
    model.fit(X_train, y_train, batch_size = batch_size,  epochs=epochs,
              validation_data=validation_data)
    
    model.save('CapsNet_COVID19.h5')
    print("Saved model to dsik")

#    from keras.preprocessing import image
#    test_image = image.load_img('img.jpg', target_size = (28, 28))
#    test_image = image.img_to_array(test_image)
#    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict_generator(X_test, y_test)
    print (model.evaluate(X_test, y_test))
    cm = confusion_matrix(y_test, result)
    print("Confusion matrix - ")
    print(cm)
    #filter = dim